In [1]:
import torch
from torch import nn
from importlib import reload
import cv2
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from prettytable import PrettyTable

import albumentations
from albumentations import augmentations
import albumentations.pytorch

In [2]:
full_df = pd.read_csv('combo_all_FULL.csv')

In [3]:
"CMFD, NIST, COVERAGE, CASIA, IMD"

def get_individual_test_df(dataframe, val):
    test_df = dataframe[dataframe['root_dir'].str.contains(val)]
    test_df = test_df[test_df["fold"].isin([1])]

    print(
        "{}: real:{}, fakes:{}".format(
            val, len(test_df[test_df["label"] == 0]), len(test_df[test_df["label"] == 1])
        )
    )
    return test_df.values

def get_test_df(dataframe):
    test_df = dataframe[dataframe["fold"].isin([1])]

    print(
        "TEST: real:{}, fakes:{}".format(
            len(test_df[test_df["label"] == 0]), len(test_df[test_df["label"] == 1])
        )
    )
    return test_df.values
    
casia_test = get_individual_test_df(full_df, "CASIA")
test_df = get_test_df(full_df)

CASIA: real:749, fakes:511
TEST: real:1353, fakes:1059


In [4]:
root_folder = "Image_Manipulation_Dataset"
def load_images(row):

    image_patch, mask_patch, label, _, ela, root_dir = row

    #------------- Load image, Ela, Mask -------------------------
    image_path = os.path.join(root_folder, root_dir, image_patch)
    ela_path = os.path.join(root_folder, root_dir, ela)

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    ela_image = cv2.imread(ela_path, cv2.IMREAD_COLOR)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    ela_image = cv2.cvtColor(ela_image, cv2.COLOR_BGR2RGB)

    if not isinstance(mask_patch, str) and np.isnan(mask_patch):
        mask_image = np.zeros((image.shape[0], image.shape[1])).astype('uint8')
    else:
        mask_path = os.path.join(root_folder, root_dir, mask_patch)
        mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if('NIST' in root_dir):
        mask_image = 255 - mask_image

    image = augmentations.geometric.functional.resize(image, 256, 256, cv2.INTER_AREA)
    mask_image = augmentations.geometric.functional.resize(mask_image, 256, 256, cv2.INTER_AREA)
    ela_image = augmentations.geometric.functional.resize(ela_image, 256, 256, cv2.INTER_AREA)

    return image, ela_image, mask_image, label



def get_tensors(image, ela_image, mask_image):

    #---------------- Reshape & Normalize -----------------------
    image = augmentations.geometric.functional.resize(image, 256, 256, cv2.INTER_AREA)
    mask_image = augmentations.geometric.functional.resize(mask_image, 256, 256, cv2.INTER_AREA)
    ela_image = augmentations.geometric.functional.resize(ela_image, 256, 256, cv2.INTER_AREA)

    normalize = {
        "mean": [0.4535408213875562, 0.42862278450748387, 0.41780105499276865],
        "std": [0.2672804038612597, 0.2550410416463668, 0.29475415579144293],
    }

    transforms_normalize = albumentations.Compose(
        [
            albumentations.Normalize(mean=normalize['mean'], std=normalize['std'], always_apply=True, p=1),
            albumentations.pytorch.transforms.ToTensorV2()
        ],
        additional_targets={'ela':'image'}
    )

    data = transforms_normalize(image=image, mask=mask_image, ela=ela_image)
    image_tensor = data["image"].unsqueeze(0)
    mask_tensor = (data["mask"] / 255.0).unsqueeze(0).unsqueeze(0)
    ela_tensor = data["ela"].unsqueeze(0)
    
    return image_tensor, ela_tensor, mask_tensor
    

In [5]:
def patch_func(img, ela, mask, patch_size):
    d = img.shape
    patches = []
    coords = []
    for i in range(0, d[0], patch_size):
        for j in range(0, d[1], patch_size):
            x = i + patch_size
            y = j + patch_size
            if x > d[0] or y > d[1]:
                break
            temp_img = img[i: x, j: y]
            temp_ela = ela[i: x, j: y]
            temp_mask = mask[i: x, j: y]
            patches.append((temp_img, temp_mask, temp_ela))
            coords.append((i, j))
    return patches, coords

In [6]:
from segmentation.merged_net import SRM_Classifer

device = 'cuda'

model_256 = SRM_Classifer()
model_256 = nn.DataParallel(model_256).to(device)
model_256.load_state_dict(torch.load('best_weights/Changed classifier+COMBO_ALL_FULLSRM+ELA_[08|03_21|22|09].h5'))

model_128 = SRM_Classifer()
model_128 = nn.DataParallel(model_128).to(device)
model_128.load_state_dict(torch.load('best_weights/COMBO_ALL_128ChangedClass_[09|03_21|11|14].h5'))

model_64 = SRM_Classifer()
model_64 = nn.DataParallel(model_64).to(device)
model_64.load_state_dict(torch.load('best_weights/(Resume 22-35-18) COMBO_ALL_64ChangedClass_[11|03_18|46|12].h5'))

# outputs = []
# def hook(module, input, output):
#     outputs.append(output)

# handle_256 = model_256.module.classifier[4].register_forward_hook(hook)
# handle_128 = model_128.module.classifier[4].register_forward_hook(hook)
# handle_64 = model_64.module.classifier[4].register_forward_hook(hook)

<All keys matched successfully>

In [7]:
def get_feat_preds(row):
    model_256.eval()
    model_128.eval()
    model_64.eval()
    
    with torch.no_grad():
        image_256, ela_image_256, mask_image_256, label = load_images(row)
        features = []
        predictions = []

        #--------- 256x256 ----------        
        image_tensor_256, ela_tensor_256, mask_tensor_256 = get_tensors(image_256, ela_image_256, mask_image_256)
        # outputs.clear()
        pred_256, inter_256, _ = model_256(image_tensor_256.to(device), ela_tensor_256.to(device))
        features.append(inter_256.cpu().detach())
        predictions.append(torch.sigmoid(pred_256.cpu().detach()))

        #--------- 128x128 ----------
        patches_128, _ = patch_func(image_256, ela_image_256, mask_image_256, 128)
        for patch in patches_128:
            image_128, mask_image_128, ela_image_128 = patch
            image_tensor_128, ela_tensor_128, mask_tensor_128 = get_tensors(image_128, ela_image_128, mask_image_128)
            
            # outputs.clear()
            pred_128, inter_128, _ = model_128(image_tensor_128.to(device), ela_tensor_128.to(device))
            features.append(inter_128.cpu().detach())
            predictions.append(torch.sigmoid(pred_128.cpu().detach()))
        
        #--------- 64x64 ----------
        patches_64, _ = patch_func(image_256, ela_image_256, mask_image_256, 64)
        for patch in patches_64:
            image_64, mask_image_64, ela_image_64 = patch
            image_tensor_64, ela_tensor_64, mask_tensor_64 = get_tensors(image_64, ela_image_64, mask_image_64)
            
            # outputs.clear()
            pred_64, inter_64, _ = model_64(image_tensor_64.to(device), ela_tensor_64.to(device))
            features.append(inter_64.cpu().detach())
            predictions.append(torch.sigmoid(pred_64.cpu().detach()))

    # torch.cuda.empty_cache()
    return features, predictions

In [8]:
def feat_pool(feat: torch.tensor, operation: str):
    """
    Fuses the image's patches feature representation
    :param feat: The network object
    :param operation: Either max or mean for the pooling operation
    :returns: The final 256-D feature representation of the entire image
    """
    if operation == "max":
        return feat.max(axis=0)
    elif operation == "mean":
        return feat.mean(axis=0)
    else:
        raise Exception("The operation can be either mean or max")

def print_preds(predictions):
    print(predictions[0].item())
    print("------")
    print(predictions[1].item(), predictions[2].item())
    print(predictions[3].item(), predictions[4].item())
    print("------")
    print(predictions[5].item(), predictions[6].item(), predictions[7].item(), predictions[8].item())
    print(predictions[9].item(), predictions[10].item(), predictions[11].item(), predictions[12].item())
    print(predictions[13].item(), predictions[14].item(), predictions[15].item(), predictions[16].item())
    print(predictions[17].item(), predictions[18].item(), predictions[19].item(), predictions[20].item())
    

In [9]:
test_preds_FULL = []
test_preds_MEAN = []
test_targets = []

In [10]:
all_features = []
all_predictions = []

In [11]:
for row in tqdm(full_df.values):

    features, predictions = get_feat_preds(row)
    
    features = torch.vstack(features)
    # condensed_feat = feat_pool(features, "mean")
    all_features.append({
        "image" : row[0],
        "label" : row[2],
        "mask" : row[1],
        "fold" : row[3],
        "root_dir" : row[-1],
        "feature" : features
    })
    # print_preds(predictions)    
    # print(torch.cat(predictions).mean())
    # print(torch.cat(predictions[1:5]).mean())
    # print(torch.cat(predictions[5:]).mean())

    all_predictions.append({
        "image" : row[0],
        "label" : row[2],
        "mask" : row[1],
        "fold" : row[3],
        "root_dir" : row[-1],
        "predictions" : [x.item() for x in predictions]
    })

    if(row[3] == 1):
        test_targets.append(row[2])
        test_preds_FULL.append(predictions[0].item())
        test_preds_MEAN.append(torch.cat(predictions).mean())

100%|██████████| 23508/23508 [4:09:52<00:00,  1.57it/s]


In [22]:
saved_features = []

In [23]:
for x in tqdm(all_features):
    name = x["root_dir"].split('/')[0] +"_"+ x["image"].split('/')[-1]
    tensor = x["feature"]
    torch.save(tensor, os.path.join('256_features', name[:-3]+"pt"))

    saved_features.append({
        "image" : x["image"],
        "label" : x["label"],
        "mask" : x["mask"],
        "fold" : x["fold"],
        "root_dir" : x["root_dir"],
        "feature" : os.path.join('256_features', name[:-3]+"pt")
    })

100%|██████████| 23508/23508 [00:17<00:00, 1347.75it/s]


In [29]:
import json
with open('all_predictions.json', 'w') as fout:
    json.dump(all_predictions , fout)

In [31]:
features_df = pd.DataFrame(saved_features)
features_df.to_csv('all_features.csv', index=False)